针对RPM产品，给定一个问题和解决方案，先扩张问题描述，然后反思这个更新的问题描述是否与原始版本保持一致。接着，根据更新的问题描述和原始的解决方案来生成多个解决方案，
再结合多个解决方案来生成最终的解决方案。目前的重点是完善问题描述和解决方案。

Use GPT-4

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


# load_dotenv("../gpt4.env")
# llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k-0613", temperature=0.7, max_tokens=4000)
load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=2000)

In [2]:
template_rewriteq = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
发明人的问题描述往往比较模糊，只集中于核心的概念和逻辑，对于问题相关的背景、价值和创新描述模糊不清。
请根据以下提示，为发明人的简单描述补充相关信息，重新生成一个更好的问题描述：
1，这个发明解决了什么具体的软件或计算问题吗？
2，那些用户、软件系统或计算环境会遇到这个问题？
3，你能举一个这个软件问题通常出现的例子吗？
4，这个问题如何影响软件系统或应用的操作、效率或用户体验？
5，目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者对用户不友好？
6，在技术领域或者行业发展中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？
7，你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？
8，这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？
9，这个软件问题是被广泛认知的，还是说你的发明揭示了它？
10，解决这个问题有没有任何特定的软件、硬件或系统约束？

发明人提供的问题描述: {raw_question}

系统的、详细的、完整的问题描述是：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [3]:
template_commentq = '''你是一个著名的产品经理，请根据发明人提供的问题描述和自动补充的问题描述，提取其中涉及到的核心概念、术语。
请提供这些概念、术语的精简描述，并说明它们之间可能存在的逻辑或者语义关系。对于复杂的技术或者过程，请提供示例来说明，以方便理解。

发明人提供的问题描述: {raw_question}
自动补全后的问题描述: {question}

概念和术语的介绍：'''

prompt_commentq = PromptTemplate(input_variables=['raw_question', 'question'], template=template_commentq)
commentq_chain = LLMChain(llm=llm, prompt=prompt_commentq, output_key='commentq')

In [4]:
template_candidates = '''你们是三个著名的程序员，请基于问题描述和用户提供的解决方案，各自提供一个具体的且有创新性的解决方案。
生成的解决方案要满足以下要求：
1，解决方案可以转换为代码设计说明书，并且方便使用代码来实现；
2，每个解决方案要包含基本的解决思路，使用的算法和数据描述，技术分析，以及实现步骤。
3，技术分析应该包括所用的算法，以及论证这个算法能够解决当前问题的分析。
4，解决步骤不应该存在多余的步骤，同时也不能缺失必须的处理过程。
5，用户提供的解决方案可以用于参考，也可以直接引用，只要最终的方案能够解决问题就可以。

待解决的问题: {question}
用户提供的解决方案：{raw_solution}

请一步步思考，确保解决方案能够解决上述问题，三个解决方案分别是：'''

prompt_candidates = PromptTemplate(input_variables=['question', 'raw_solution'], template=template_candidates)
candidates_chain = LLMChain(llm=llm, prompt=prompt_candidates, output_key='candidates')

In [5]:
template_rewritea = '''你是一个在知名大公司工作的软件系统架构师，你的任务是根据问题描述和多个解决方案，重新实现一个详细的、现实可行的实现方案。
这个解决方案需要满足以下条件：
1，它是可实现的，能够根据它来写出对应的代码；同时，它有创新性，可以用于专利申请；
2，它不需要考虑所有的可能性，只要能够解决最核心的问题即可。
3，请描述它的基本思路是什么？用到了哪些成熟算法和数据？
4，请具体分析这个解决思路是如何解决这个问题的。
5，它与一般的解决方法有什么不同？不管是算法上的不同，应用场景的不同，还是使用的数据不同都可以。
6，创新点对应的优势和价值是什么？如果创新点的价值不高，可以通过什么样的方法来提高？
7，它的具体解决步骤是什么？请以发明人的解决方案为核心思路，扩展实现步骤，并且保证每个步骤都是必须的。
8，如果自动生成的方案中对某一问题的解决方法更好，请将这个更好的方法加入到解决方案中。

问题描述: {question}
自动生成的解决方案: {candidates}
发明人提供的解决方案：{raw_solution}

请重新生成一个具备创新性的解决方案：'''

prompt_rewritea = PromptTemplate(input_variables=['question', 'candidates', 'raw_solution'], template=template_rewritea)
rewritea_chain = LLMChain(llm=llm, prompt=prompt_rewritea, output_key='solution')

In [6]:
template_commenta = '''你是一个著名的程序员，请根据问题描述和选定的解决方案，提供一个详细的代码实现说明书。
请根据以下提示，生成一份代码实现说明：
1，可以根据这个解决方案来实现相应的功能吗？如果不能，请给出理由，并补充解决方法；
2，实现这个解决方案都需要哪些数据？评估获取这些数据的工作量；
3，在解决方案的执行步骤中，有哪些步骤并不是必要的，或者它们创造的价值小于提供相应数据的代价；
4，针对解决方案的必须步骤，提供具体的实现说明；若是某个步骤过于复杂，请将它拆分为多个子步骤后，再提供实现说明；
5，在涉及算法时，请给出具体的算法名；在涉及到数据时，请提供一个示例；

问题描述: {question}
解决方案: {solution}

针对解决方案的说明：'''

prompt_commenta = PromptTemplate(input_variables=['question', 'solution'], template=template_commenta)
commenta_chain = LLMChain(llm=llm, prompt=prompt_commenta, output_key='commenta')

In [7]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，帮助专利发明人设计发明点设计，并起草专利草稿。请根据以下待解决问题及解决方案的描述，起草专利的背景介绍部分。
专利的背景介绍要包含所解决问题所在的技术领域介绍，问题的简单介绍，以及这个问题所产生的不良影响。
待解决问题的描述: {question}
解决方案的描述: {solution}
请生成这个专利发明点的背景介绍（总长度不超过2000字）：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [8]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求（总长度不超过4000字）：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [9]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明（总长度不超过2000字）：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [10]:
from langchain.chains import SequentialChain

def generate(pno, question, solution, version=1):
    result = overall_chain({'raw_question': question, 'raw_solution': solution})

    with open(f'patents/patent-rpm-{pno}v{version}.txt', 'w') as f:
        f.write('>>> original question: \n' + result['raw_question'])  
        f.write('\n\n>>> original solution: \n' + result['raw_solution'])
        f.write('\n\n>>> revised question: \n' + result['question'])
        f.write('\n\n>>> commented question: \n' + result['commentq'])
        f.write('\n\n>>> solution candidates: \n' + result['candidates'])
        f.write('\n\n>>> final solution: \n' + result['solution'])
        f.write('\n\n>>> commented solution: \n' + result['commenta'])
        f.write('\n\n>>> background: \n' + result['background'])
        f.write('\n\n>>> claims: \n' + result['claims'])
        f.write('\n\n>>> explanation: \n' + result['explanation'])

overall_chain = SequentialChain(
    chains=[rewriteq_chain, commentq_chain, candidates_chain, rewritea_chain, commenta_chain, background_chain, claims_chain, explanation_chain],
    input_variables=["raw_question", "raw_solution"],
    # Here we return multiple variables
    output_variables=["question", "commentq", "candidates", "solution", "commenta", "background", "claims", "explanation"],
    verbose=True)

In [11]:
import pandas as pd


version = 7
df = pd.read_excel('patents/rpm-patents-0628d.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    print(pno, question)
    generate(pno, question, solution, version)

P73 使用ChatGPT可以生成能够正常运行的、不太复杂的代码，可以用于生成RPA代码（模拟人工操作来实现UI任务的自动化执行）。一般的方法是设置一个比较详细的生成提示词，让它直接生成最终代码，但是这样的生成过程难以控制，比如使用了不存在的库函数，或者缺少必要的数据等等，也难以保证生成的流程可以正确执行。为了保证实现方法足够简单，在提供解决方案时，除了chatgpt，请不要使用任何别的机器学习或者深度学习方法。


> Entering new SequentialChain chain...

> Finished chain.
P74 使用ChatGPT可以生成能够正常运行的不太复杂的代码，但是没有简单的方法控制生成的代码。在实际业务场景中，一段代码只能够使用数量有限的几个数据，以及调用一定范围内的函数，而任意生成的代码则不考虑这种限制，所以生成的代码即使可以正确运行，也无法直接使用，还需要程序员重新理解代码、修改代码，并没有节省太多人力。为了保证实现方法足够简单，在提供解决方案时，除了chatgpt，请不要使用任何别的机器学习或者深度学习方法。


> Entering new SequentialChain chain...

> Finished chain.
P75 使用ChatGPT来生成代码时，生成所需要的上下文有可能非常多，无法容纳在模型的窗口大小范围之内。常用的方法是先对上下文做摘要，然后使用摘要作为上下文来指导代码的生成。由于代码包含的细节非常多，这种摘要方式很容易丢失大量的细节。为了保证实现方法足够简单，在提供解决方案时，除了chatgpt，请不要使用任何别的机器学习或者深度学习方法。


> Entering new SequentialChain chain...

> Finished chain.
